# Part A

In [1]:
#Imporing libraries
import os
import pandas as pd
import numpy as np
import statistics as stat
import itertools
from functools import reduce

In [43]:
import session_info
session_info.show()

### Data Preprocessing

In [2]:
#Loading ratings data
baseDir = 'D:\\Recommdar_system\\Assignment1\\ml-latest-small'
ratings_df = pd.read_csv(baseDir+"\\ratings.csv")

ratings_df.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# Droping 'timestamp' column
ratings_df.drop('timestamp', inplace=True, axis=1)

ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
#Loading movies data
movies_df = pd.read_csv(baseDir+'\\movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


#### Preproceesing the movie data

In [5]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year']

0       (1995)
1       (1995)
2       (1995)
3       (1995)
4       (1995)
         ...  
9737    (2017)
9738    (2017)
9739    (2017)
9740    (2018)
9741    (1991)
Name: year, Length: 9742, dtype: object

In [6]:
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['year']

0       1995
1       1995
2       1995
3       1995
4       1995
        ... 
9737    2017
9738    2017
9739    2017
9740    2018
9741    1991
Name: year, Length: 9742, dtype: object

In [7]:
movies_df['title']

0                                Toy Story (1995)
1                                  Jumanji (1995)
2                         Grumpier Old Men (1995)
3                        Waiting to Exhale (1995)
4              Father of the Bride Part II (1995)
                          ...                    
9737    Black Butler: Book of the Atlantic (2017)
9738                 No Game No Life: Zero (2017)
9739                                 Flint (2017)
9740          Bungo Stray Dogs: Dead Apple (2018)
9741          Andrew Dice Clay: Dice Rules (1991)
Name: title, Length: 9742, dtype: object

In [8]:
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title']

C:\Users\HP\AppData\Local\Temp\ipykernel_17612\2532954281.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


0                                Toy Story 
1                                  Jumanji 
2                         Grumpier Old Men 
3                        Waiting to Exhale 
4              Father of the Bride Part II 
                       ...                 
9737    Black Butler: Book of the Atlantic 
9738                 No Game No Life: Zero 
9739                                 Flint 
9740          Bungo Stray Dogs: Dead Apple 
9741          Andrew Dice Clay: Dice Rules 
Name: title, Length: 9742, dtype: object

In [9]:
#Applying the strip function to remove ending whitespace 
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [10]:
# Drop 'genres' column 
movies_df.drop('genres', inplace=True, axis=1)
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


## Implementing user-based collaborative filtering approach for group

After preprocessing, the process we will follow for creating a User Based recommendation system (group):

* To implement a user based collaborative filtering approach, at first we select a few users. For example, here we choose 3 user having a userId of 100, 200, 300. 
* Based on our selected group rating to movies, have to find the top X neighbours
* Will generate sub dataframes depending on the group's users and sort them, giving precedence to the users who have the most things in common with the group's users.
* Get the watched movie record of the user for each neighbour group.
* Calculate a similarity score using a formula. Here, we will use (Pearson Correlation).
* Recommend the items with the highest score


In [11]:
# Create a group of 3 users: 100, 200 , 300
choosen_group = [55,155,255]

In [12]:
# Getting all the movies for each user of the group as seen or rated
inputMovies=[]

for user in choosen_group:
    user_Id = user
    extracted_movies = ratings_df[ratings_df['userId']== user_Id]
    inputMovies.append(extracted_movies)


In [13]:
for i in range(len(inputMovies)):
    print("Number of movies rated",len(inputMovies[i]))
    print(inputMovies[i].head())

Number of movies rated 25
      userId  movieId  rating
7964      55      186     0.5
7965      55      673     0.5
7966      55     1275     4.0
7967      55     1293     4.0
7968      55     1357     0.5
Number of movies rated 46
       userId  movieId  rating
22638     155        1     3.0
22639     155      141     3.0
22640     155      153     3.0
22641     155      260     4.0
22642     155      333     4.0
Number of movies rated 44
       userId  movieId  rating
37682     255       34     1.0
37683     255      223     2.0
37684     255      344     5.0
37685     255      420     5.0
37686     255      588     2.0


In [14]:
# Finding the useres who rates similar movies as our users of the group

similarTypeOfUsers = []
for i in range(len(inputMovies)):
    similarTypeOfUsers_temp = ratings_df[ratings_df['movieId'].isin(inputMovies[i]['movieId'].tolist())]
    similarTypeOfUsers.append(similarTypeOfUsers_temp)
    
for i in range(len(similarTypeOfUsers)):
    print(similarTypeOfUsers[i].head())

     userId  movieId  rating
39        1      673     3.0
86        1     1275     5.0
118       1     2005     5.0
156       1     2427     5.0
165       1     2542     5.0
    userId  movieId  rating
0        1        1     4.0
15       1      260     5.0
18       1      333     5.0
33       1      592     4.0
92       1     1377     3.0
     userId  movieId  rating
12        1      223     3.0
73        1     1210     5.0
77        1     1220     5.0
85        1     1270     5.0
165       1     2542     5.0


In [15]:
# Creating sub-dataframes based on users of the group

userSubsetGroup = []
for i in range(len(similarTypeOfUsers)):
    userSubsetGroup_temp = similarTypeOfUsers[i].groupby(['userId'])
    userSubsetGroup.append(userSubsetGroup_temp)

In [16]:
# Sortng the dataframe
for i in range(len(userSubsetGroup)): 
    userSubsetGroup[i] = sorted(userSubsetGroup[i], key= lambda x:len(x[1]), reverse = True)
    
# As 1st one is the same as our selected user, let's see the 2nd top user for each user of the group
for i in range(len(userSubsetGroup)):
    print(userSubsetGroup[i][1])


(414,        userId  movieId  rating
62577     414      673     2.0
62770     414     1275     4.0
62780     414     1293     3.0
62796     414     1357     4.0
62947     414     1947     4.0
62966     414     2005     5.0
63013     414     2100     3.0
63073     414     2278     4.0
63109     414     2393     3.0
63121     414     2407     4.0
63133     414     2427     4.0
63156     414     2490     3.0
63166     414     2542     5.0
63176     414     2580     5.0
63225     414     2763     2.0
63261     414     2890     5.0
63321     414     3101     4.0
63379     414     3275     5.0
63573     414     4011     5.0
64374     414    27831     4.0
64404     414    33166     4.0
64502     414    44665     4.0
64562     414    48516     4.5
64643     414    54286     4.0)
(414,        userId  movieId  rating
62294     414        1     4.0
62352     414      141     4.0
62358     414      153     2.0
62410     414      260     5.0
62439     414      333     3.0
62484     414      432    

C:\Users\HP\AppData\Local\Temp\ipykernel_17612\2123221467.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup[i] = sorted(userSubsetGroup[i], key= lambda x:len(x[1]), reverse = True)
C:\Users\HP\AppData\Local\Temp\ipykernel_17612\2123221467.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup[i] = sorted(userSubsetGroup[i], key= lambda x:len(x[1]), reverse = True)
C:\Users\HP\AppData\Local\Temp\ipykernel_17612\2123221467.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single 

In [17]:
# number of users who rated all or a selection of the movies in the group along with the original user.
for i in range(len(userSubsetGroup)):
    print(len(userSubsetGroup[i]))

345
558
493


We will select a subset of users to iterate through. This limit is imposed because we don’t want to waste too much time going through every single user.

In [18]:
for i in range(len(userSubsetGroup)):
    userSubsetGroup[i] = userSubsetGroup[i][1:101]
    print(len(userSubsetGroup[i]))

100
100
100


In [19]:
# List of correlations of the users of the group
pearsonCorrelation = []

for i in range(len(userSubsetGroup)):
    # Storing the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
    pearsonCorrelationDict = {}
    
    # Get movies list of a user of the group and sort them by movieId
    temp_inputMovies = inputMovies[i].sort_values(by='movieId')
    
    #For every user group in our subset
    for name, group in userSubsetGroup[i]:
        #Let's start by sorting the input and current user group so the values aren't mixed up later on
        group = group.sort_values(by='movieId')
        #Get the N for the formula
        nRatings = len(group)
        #Get the review scores for the movies that they both have in common
        temp_df = temp_inputMovies[temp_inputMovies['movieId'].isin(group['movieId'].tolist())]
        #And then store them in a temporary buffer variable in a list format to facilitate future calculations
        tempRatingList = temp_df['rating'].tolist()
        #Let's also put the current user group reviews in a list format
        tempGroupList = group['rating'].tolist()
        
#         scipy.stats import pearsonr
#         pearsonr(tempRatingList,tempGroupList)[0]
        #Now let's calculate the pearson correlation between two users, so called, x and y
        Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
        Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
        Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

        #If the denominator is different than zero, then divide, else, 0 correlation.
        if Sxx != 0 and Syy != 0:
            pearsonCorrelationDict[name] = Sxy/np.sqrt(Sxx*Syy)
        else:
            pearsonCorrelationDict[name] = 0                 
    
    pearsonCorrelation.append(pearsonCorrelationDict)

In [20]:
# Inspecting the Correlation Values
for i in range(len(pearsonCorrelation)):
    print(pearsonCorrelation[i].items())
    print(" ")

dict_items([(414, 0.440988931140743), (599, 0.13718452337495202), (68, -0.26128935153535743), (448, 0.48907743623896527), (590, 0.5219354722663045), (28, -0.13633510993198109), (219, 0.33633639699815715), (274, 0.0), (438, 0.6727731263221208), (480, 0.529014483509057), (64, 0.08549369507247383), (182, -0.11345948433149503), (288, -0.05022924559450697), (474, 0.1526704643232989), (608, 0.4483254534703861), (18, 0.7747080984269888), (307, 0.6949387692979541), (380, 0.35661843149344546), (387, 0.567273758346191), (483, 0.6393443138580434), (42, 0.34800887433944544), (177, 0.5514308221712151), (217, 0.04999999999999998), (477, 0.6075727502438608), (561, 0.6291914187186425), (580, 0.454706833791235), (603, -0.1966571461940076), (19, 0.3857122687381099), (140, -0.3948468805000098), (226, 0.576129050610196), (239, 0.8418567021412103), (249, 0.5944400023051198), (45, 0.48617494035064146), (51, -0.06401435500123971), (57, 0.8386278693775346), (305, 0.25474621817335047), (318, 0.5957783116811052

In [21]:
pearsonDF =[]
for i in range(len(pearsonCorrelation)):
    pearsonDF_temp = pd.DataFrame.from_dict(pearsonCorrelation[i], orient='index')
    pearsonDF.append(pearsonDF_temp)

# Print
for i in range(len(pearsonDF)):
    print(pearsonDF[i].head())

            0
414  0.440989
599  0.137185
68  -0.261289
448  0.489077
590  0.521935
            0
414 -0.030167
599  0.112348
608  0.446487
182 -0.357009
68   0.193540
            0
414 -0.158765
474 -0.513479
599 -0.067984
288 -0.391316
448 -0.159034


In [22]:
#Similarity score of other similar users to our selected user (100, 200, 300)
for i in range(len(pearsonDF)):
    pearsonDF[i].columns = ['similarityIndex']
    pearsonDF[i]['userId'] = pearsonDF[i].index
    pearsonDF[i].index = range(len(pearsonDF[i]))

# Print
for i in range(len(pearsonDF)):
    print(pearsonDF[i].head())

   similarityIndex  userId
0         0.440989     414
1         0.137185     599
2        -0.261289      68
3         0.489077     448
4         0.521935     590
   similarityIndex  userId
0        -0.030167     414
1         0.112348     599
2         0.446487     608
3        -0.357009     182
4         0.193540      68
   similarityIndex  userId
0        -0.158765     414
1        -0.513479     474
2        -0.067984     599
3        -0.391316     288
4        -0.159034     448


Now let’s get the top users that are most similar to the input

In [23]:
# Top Similar users
topUsers = []
for i in range(len(pearsonDF)):
    topUsers_temp=pearsonDF[i].sort_values(by='similarityIndex', ascending=False)
    topUsers.append(topUsers_temp)
    
# Print
for i in range(len(topUsers)):
    print(topUsers[i].head(5))

    similarityIndex  userId
99         0.945651     282
76         0.910182     317
41         0.909857     232
42         0.897161     313
98         0.892699     203
    similarityIndex  userId
96         0.765478      93
81         0.625485     522
41         0.624204     580
32         0.615578      82
2          0.446487     608
    similarityIndex  userId
96         0.694616     200
26         0.518163     294
88         0.479349     232
71         0.447136     607
98         0.434428     367


The movies that the users in our pearsonDF have watched must first be obtained from the ratings dataframe, and we must then store their correlation in a new column named similarityIndex.

In [24]:
topUsersRating =[]
for i in range(len(topUsers)):
    topUsersRating_temp = topUsers[i].merge(ratings_df, left_on='userId', right_on='userId', how='inner')
    topUsersRating.append(topUsersRating_temp)
    
# Print
for i in range(len(topUsersRating)):  
    print(topUsersRating[i].head())

   similarityIndex  userId  movieId  rating
0         0.945651     282        1     4.5
1         0.945651     282        6     4.0
2         0.945651     282       10     3.5
3         0.945651     282       16     4.0
4         0.945651     282       25     4.0
   similarityIndex  userId  movieId  rating
0         0.765478      93        1     3.0
1         0.765478      93        2     5.0
2         0.765478      93       10     4.0
3         0.765478      93       15     4.0
4         0.765478      93       34     5.0
   similarityIndex  userId  movieId  rating
0         0.694616     200        1     3.5
1         0.694616     200        5     4.0
2         0.694616     200       10     4.5
3         0.694616     200       19     3.5
4         0.694616     200       34     2.5


In [25]:
# Calculating the mean ratings of all the similar users

meanRating = []

for i in range(len(topUsersRating)):
    meanRating_temp = topUsersRating[i].groupby('userId').mean()[['rating']]
    meanRating_temp.columns = ['meanRating']
    meanRating_temp['userId'] = meanRating_temp.index
    meanRating_temp.index = range(len(meanRating_temp))
    
    meanRating.append(meanRating_temp)

# Inspecting...
for i in range(len(meanRating)):
    print(meanRating[i].head())

   meanRating  userId
0    4.366379       1
1    3.732072      18
2    2.607397      19
3    3.548148      27
4    3.020175      28
   meanRating  userId
0    4.366379       1
1    3.230263       7
2    2.607397      19
3    3.260722      21
4    3.020175      28
   meanRating  userId
0    3.732072      18
1    2.607397      19
2    3.260722      21
3    3.548148      27
4    3.565909      42


In [26]:
# Merging the top similar users average rating score 
for i in range(len(topUsersRating)):
    topUsersRating[i] = topUsersRating[i].merge(meanRating[i], left_on='userId', right_on='userId', how='inner')

# Inspecting...  
for i in range(len(topUsersRating)):
    print(topUsersRating[i].head())

   similarityIndex  userId  movieId  rating  meanRating
0         0.945651     282        1     4.5    4.033755
1         0.945651     282        6     4.0    4.033755
2         0.945651     282       10     3.5    4.033755
3         0.945651     282       16     4.0    4.033755
4         0.945651     282       25     4.0    4.033755
   similarityIndex  userId  movieId  rating  meanRating
0         0.765478      93        1     3.0     4.28866
1         0.765478      93        2     5.0     4.28866
2         0.765478      93       10     4.0     4.28866
3         0.765478      93       15     4.0     4.28866
4         0.765478      93       34     5.0     4.28866
   similarityIndex  userId  movieId  rating  meanRating
0         0.694616     200        1     3.5     3.80988
1         0.694616     200        5     4.0     3.80988
2         0.694616     200       10     4.5     3.80988
3         0.694616     200       19     3.5     3.80988
4         0.694616     200       34     2.5     

#### Now all we need to do is simply multiply the movie rating by its weight (the similarity index), then sum up the new ratings and divide it by the sum of the weights.

In [27]:
#Multiplyinh the similarity by the user's ratings
for i in range(len(topUsersRating)):
    topUsersRating[i]['weightedRating'] = topUsersRating[i]['similarityIndex']*(topUsersRating[i]['rating']-topUsersRating[i]['meanRating'])

# Print
for i in range(len(topUsersRating)):
    print(topUsersRating[i].head())

   similarityIndex  userId  movieId  rating  meanRating  weightedRating
0         0.945651     282        1     4.5    4.033755        0.440905
1         0.945651     282        6     4.0    4.033755       -0.031921
2         0.945651     282       10     3.5    4.033755       -0.504746
3         0.945651     282       16     4.0    4.033755       -0.031921
4         0.945651     282       25     4.0    4.033755       -0.031921
   similarityIndex  userId  movieId  rating  meanRating  weightedRating
0         0.765478      93        1     3.0     4.28866       -0.986441
1         0.765478      93        2     5.0     4.28866        0.544515
2         0.765478      93       10     4.0     4.28866       -0.220963
3         0.765478      93       15     4.0     4.28866       -0.220963
4         0.765478      93       34     5.0     4.28866        0.544515
   similarityIndex  userId  movieId  rating  meanRating  weightedRating
0         0.694616     200        1     3.5     3.80988       -0

In [28]:

sumOfTopUsersWeightRating = []

for i in range(len(topUsersRating)):
    topUsersRating_temp = topUsersRating[i].groupby('movieId').sum()[['weightedRating']]
    topUsersRating_temp.columns = ['sum_weightedRatingScore']
    topUsersRating_temp['movieId'] = topUsersRating_temp.index
    
    sumOfTopUsersWeightRating.append(topUsersRating_temp)

# Inspecting...    
for i in range(len(sumOfTopUsersWeightRating)):  
    print(sumOfTopUsersWeightRating[i].head())

         sum_weightedRatingScore  movieId
movieId                                  
1                       9.833063        1
2                       0.459007        2
3                      -0.301404        3
4                      -0.378737        4
5                      -2.416621        5
         sum_weightedRatingScore  movieId
movieId                                  
1                      -3.430038        1
2                       0.409365        2
3                      -1.812607        3
4                      -0.256447        4
5                      -0.492905        5
         sum_weightedRatingScore  movieId
movieId                                  
1                      -2.008994        1
2                       2.205751        2
3                      -1.482757        3
4                       0.354875        4
5                       2.249897        5


In [29]:
recommendation_df = []

for i in range(len(sumOfTopUsersWeightRating)):
    # Create an empty dataframe
    recommendation__temp_df = pd.DataFrame()
    
    # Now we take the weighted average
    meanRatings = inputMovies[i]['rating'].mean()
    recommendation__temp_df['weightedAverageRecommendationScore'] = meanRatings+(sumOfTopUsersWeightRating[i]['sum_weightedRatingScore']/topUsers[i]['similarityIndex'].sum())
    recommendation__temp_df['movieId'] = recommendation__temp_df.index
    
    # Removing the name of the index for future ambiguity
    recommendation__temp_df.index.name = None
    # Swaping Coumns
    titles = list(recommendation__temp_df)
    titles[0], titles[1] = titles[1], titles[0]
    recommendation__temp_df = recommendation__temp_df[titles]
    
    recommendation_df.append(recommendation__temp_df)

# Print
for i in range(len(recommendation_df)):    
    print(recommendation_df[i].head())

   movieId  weightedAverageRecommendationScore
1        1                            3.167984
2        2                            2.855310
3        3                            2.829947
4        4                            2.827367
5        5                            2.759393
   movieId  weightedAverageRecommendationScore
1        1                            3.005972
2        2                            3.680629
3        3                            3.290186
4        4                            3.563633
5        5                            3.522083
   movieId  weightedAverageRecommendationScore
1        1                            2.748157
2        2                            2.370580
3        3                            2.701014
4        4                            2.536390
5        5                            2.366625


#### Sorting and see the top 20 recommended movies

In [30]:
for i in range(len(recommendation_df)):
    recommendation_df[i] = recommendation_df[i].sort_values(by='weightedAverageRecommendationScore', ascending=False)
    recommendation_df[i] = recommendation_df[i].reset_index(drop=True)
    recommendation_df[i]['rank'] = recommendation_df[i].index
    
    print(recommendation_df[i].head(20))

    movieId  weightedAverageRecommendationScore  rank
0       296                            3.683337     0
1      2959                            3.628818     1
2       318                            3.608325     2
3      1196                            3.577789     3
4       356                            3.569874     4
5        50                            3.548545     5
6      2571                            3.516371     6
7     48516                            3.512260     7
8       260                            3.506657     8
9       858                            3.502398     9
10     2028                            3.494954    10
11     1197                            3.461974    11
12     2858                            3.423698    12
13       47                            3.409213    13
14     1136                            3.394028    14
15     1198                            3.391485    15
16      527                            3.377268    16
17     1221                 

In [31]:
# Top 20 Recommended Movies
for i in range(len(recommendation_df)):
    print("User :", i)
    print(movies_df.loc[movies_df['movieId'].isin(recommendation_df[i].head(20)['movieId'].tolist())])

User : 0
      movieId                                              title  year
43         47                               Seven (a.k.a. Se7en)  1995
46         50                                Usual Suspects, The  1995
224       260                 Star Wars: Episode IV - A New Hope  1977
257       296                                       Pulp Fiction  1994
277       318                          Shawshank Redemption, The  1994
314       356                                       Forrest Gump  1994
461       527                                   Schindler's List  1993
659       858                                     Godfather, The  1972
863      1136                    Monty Python and the Holy Grail  1975
898      1196     Star Wars: Episode V - The Empire Strikes Back  1980
899      1197                                Princess Bride, The  1987
900      1198  Raiders of the Lost Ark (Indiana Jones and the...  1981
922      1221                            Godfather: Part II, The  19

In [32]:
# Combining all users ratings by movieId
recommendation_df_group = reduce(lambda left,right: pd.merge(left,right,on='movieId'), recommendation_df)
recommendation_df_group.head(20)

,movieId,weightedAverageRecommendationScore_x,rank_x,weightedAverageRecommendationScore_y,rank_y,weightedAverageRecommendationScore,rank
0,296,3.683337,0,4.518566,9,3.144336,15
1,2959,3.628818,1,4.306878,17,2.884375,77
2,318,3.608325,2,3.875708,119,3.180874,9
3,1196,3.577789,3,4.613628,7,2.917234,61
4,356,3.569874,4,3.762246,334,2.709158,249
5,50,3.548545,5,3.923615,83,2.944849,51
6,2571,3.516371,6,4.625165,6,2.837294,98
7,48516,3.512260,7,3.984167,58,2.641640,667
8,260,3.506657,8,4.735162,3,2.724278,217
9,858,3.502398,9,4.200456,25,3.504008,0


## First aggregation approach (The average method)

In [33]:
# Calculate averaging score and sorting the data
recommendation_df_average_method = recommendation_df_group.copy(deep = False)
recommendation_df_average_method['AverageGroupRecommendationScore']= recommendation_df_average_method[['weightedAverageRecommendationScore_x', 'weightedAverageRecommendationScore_y', 'weightedAverageRecommendationScore']].mean(axis=1)
recommendation_df_average_method = recommendation_df_average_method.sort_values(by=['AverageGroupRecommendationScore'], ascending=False)

recommendation_df_average_method = recommendation_df_average_method.reset_index(drop=True)
recommendation_df_average_method['final_rank'] = recommendation_df_average_method.index

recommendation_df_average_method.head(20)

,movieId,weightedAverageRecommendationScore_x,rank_x,weightedAverageRecommendationScore_y,rank_y,weightedAverageRecommendationScore,rank,AverageGroupRecommendationScore,final_rank
0,296,3.683337,0,4.518566,9,3.144336,15,3.782080,0
1,2028,3.494954,10,4.936516,0,2.792171,128,3.741214,1
2,858,3.502398,9,4.200456,25,3.504008,0,3.735621,2
3,1196,3.577789,3,4.613628,7,2.917234,61,3.702884,3
4,2571,3.516371,6,4.625165,6,2.837294,98,3.659610,4
5,260,3.506657,8,4.735162,3,2.724278,217,3.655366,5
6,1198,3.391485,15,4.413992,13,3.147305,14,3.650928,6
7,2959,3.628818,1,4.306878,17,2.884375,77,3.606690,7
8,1136,3.394028,14,3.928674,78,3.447509,2,3.590070,8
9,318,3.608325,2,3.875708,119,3.180874,9,3.554969,9


In [34]:
##  The 20 movies with the highest prediction scores that (i) the average method suggests

In [35]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df_average_method.head(20)['movieId'].tolist())]

,movieId,title,year
31,32,Twelve Monkeys (a.k.a. 12 Monkeys),1995
43,47,Seven (a.k.a. Se7en),1995
46,50,"Usual Suspects, The",1995
224,260,Star Wars: Episode IV - A New Hope,1977
257,296,Pulp Fiction,1994
277,318,"Shawshank Redemption, The",1994
461,527,Schindler's List,1993
520,608,Fargo,1996
659,858,"Godfather, The",1972
863,1136,Monty Python and the Holy Grail,1975


## The second aggregation method  (The least misery method)

In [36]:
# Calculating least misery score and sorting the data
recommendation_df_leastMisery_method = recommendation_df_group.copy(deep = False)
recommendation_df_leastMisery_method['MinimumGroupRecommendationScore']= recommendation_df_leastMisery_method[['weightedAverageRecommendationScore_x', 'weightedAverageRecommendationScore_y', 'weightedAverageRecommendationScore']].min(axis=1)
recommendation_df_leastMisery_method = recommendation_df_leastMisery_method.sort_values(by=['MinimumGroupRecommendationScore'], ascending=False)

recommendation_df_leastMisery_method= recommendation_df_leastMisery_method.reset_index(drop=True)
recommendation_df_leastMisery_method['final_rank'] = recommendation_df_leastMisery_method.index

recommendation_df_leastMisery_method.head(20)

,movieId,weightedAverageRecommendationScore_x,rank_x,weightedAverageRecommendationScore_y,rank_y,weightedAverageRecommendationScore,rank,MinimumGroupRecommendationScore,final_rank
0,858,3.502398,9,4.200456,25,3.504008,0,3.502398,0
1,1136,3.394028,14,3.928674,78,3.447509,2,3.394028,1
2,2858,3.423698,12,3.855983,144,3.346250,4,3.346250,2
3,1221,3.373737,17,3.754803,363,3.250486,6,3.250486,3
4,608,3.210705,41,3.871212,125,3.360358,3,3.210705,4
5,750,3.221935,37,3.650648,1468,3.209790,7,3.209790,5
6,318,3.608325,2,3.875708,119,3.180874,9,3.180874,6
7,1213,3.319685,22,3.645411,1588,3.157551,12,3.157551,7
8,1198,3.391485,15,4.413992,13,3.147305,14,3.147305,8
9,296,3.683337,0,4.518566,9,3.144336,15,3.144336,9


##  The 20 movies with the highest prediction scores that (ii) the least misery method suggests

In [37]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df_leastMisery_method.head(20)['movieId'].tolist())]

,movieId,title,year
257,296,Pulp Fiction,1994
277,318,"Shawshank Redemption, The",1994
461,527,Schindler's List,1993
510,593,"Silence of the Lambs, The",1991
520,608,Fargo,1996
602,750,Dr. Strangelove or: How I Learned to Stop Worr...,1964
659,858,"Godfather, The",1972
686,904,Rear Window,1954
828,1089,Reservoir Dogs,1992
863,1136,Monty Python and the Holy Grail,1975


# Part B
### Proposed method:
* Most pleasure strategy. It works as the least misery strategy, but instead of considering for an item the smallest ratings of the users, it selects the greatest ones. It belongs to the Borderline strategies method. Below is an example: 
| Name     | item 1 | item 2    |
| :---        |    :----:   |          ---: |
| Md      | 3       | 5  |
| Maruf   | 2        | 8      |
| Rayhan   | 7        | 7      |

Considers the maximum rating of an item. Group rating for  item will be 8 since it’s the largest amongst(5,8,7)


In [38]:
# Calculating Max score and sorting the data
recommendation_df_max_method = recommendation_df_group.copy(deep = False)
recommendation_df_max_method['MaxGroupRecommendationScore']= recommendation_df_max_method[['weightedAverageRecommendationScore_x', 'weightedAverageRecommendationScore_y', 'weightedAverageRecommendationScore']].max(axis=1)
recommendation_df_max_method = recommendation_df_max_method.sort_values(by=['MaxGroupRecommendationScore'], ascending=False)

recommendation_df_max_method = recommendation_df_max_method.reset_index(drop=True)
recommendation_df_max_method['final_rank'] = recommendation_df_max_method.index

recommendation_df_average_method.tail(20)


,movieId,weightedAverageRecommendationScore_x,rank_x,weightedAverageRecommendationScore_y,rank_y,weightedAverageRecommendationScore,rank,AverageGroupRecommendationScore,final_rank
8020,3243,2.617693,8430,3.048643,8350,2.501269,7824,2.722535,8020
8021,2643,2.664365,8386,3.197344,8330,2.304356,8551,2.722022,8021
8022,648,2.692977,8339,3.482611,7883,1.966494,8616,2.714028,8022
8023,2082,2.625769,8424,3.039203,8353,2.459381,8202,2.708118,8023
8024,1690,2.638521,8415,3.147909,8338,2.336203,8521,2.707544,8024
8025,1499,2.493522,8475,3.291684,8293,2.324299,8532,2.703168,8025
8026,2384,2.791128,7670,2.784776,8361,2.520473,7493,2.698792,8026
8027,762,2.601438,8442,3.275220,8303,2.219281,8591,2.698646,8027
8028,466,2.561534,8461,3.120478,8343,2.405488,8404,2.695833,8028
8029,420,2.636150,8417,3.378364,8203,2.027046,8612,2.680520,8029


In [39]:
recommendation_df_disagreement_max = pd.DataFrame()

weights_x = abs(recommendation_df_max_method['weightedAverageRecommendationScore_x']-recommendation_df_max_method['MaxGroupRecommendationScore'])
weights_y = abs(recommendation_df_max_method['weightedAverageRecommendationScore_y']-recommendation_df_max_method['MaxGroupRecommendationScore'])
weights = abs(recommendation_df_max_method['weightedAverageRecommendationScore']-recommendation_df_max_method['MaxGroupRecommendationScore'])


for i in range(len(recommendation_df_max_method)):
    if (weights_x[i] <=3.5 and weights_y[i] <=3.5 and weights[i] <=3.5):
        recommendation_df_disagreement_max=recommendation_df_disagreement_max.append(recommendation_df_max_method.loc[i], ignore_index=True)

recommendation_df_disagreement_max.head(20)

C:\Users\HP\AppData\Local\Temp\ipykernel_17612\3955060406.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendation_df_disagreement_max=recommendation_df_disagreement_max.append(recommendation_df_max_method.loc[i], ignore_index=True)


,movieId,weightedAverageRecommendationScore_x,rank_x,weightedAverageRecommendationScore_y,rank_y,weightedAverageRecommendationScore,rank,MaxGroupRecommendationScore,final_rank
0,2028.0,3.494954,10.0,4.936516,0.0,2.792171,128.0,4.936516,0.0
1,2628.0,2.666014,8382.0,4.925712,1.0,2.039024,8611.0,4.925712,1.0
2,1210.0,3.337745,21.0,4.742338,2.0,2.325765,8530.0,4.742338,2.0
3,260.0,3.506657,8.0,4.735162,3.0,2.724278,217.0,4.735162,3.0
4,3578.0,3.360991,18.0,4.686289,4.0,2.304362,8550.0,4.686289,4.0
5,6934.0,2.649415,8405.0,4.625742,5.0,2.469484,8141.0,4.625742,5.0
6,2571.0,3.516371,6.0,4.625165,6.0,2.837294,98.0,4.625165,6.0
7,1196.0,3.577789,3.0,4.613628,7.0,2.917234,61.0,4.613628,7.0
8,5378.0,2.733336,8228.0,4.551882,8.0,2.480560,8044.0,4.551882,8.0
9,296.0,3.683337,0.0,4.518566,9.0,3.144336,15.0,4.518566,9.0


# Top 20 Recommended Movies after considering disagrement

In [40]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df_disagreement_max.head(20)['movieId'].tolist())]

,movieId,title,year
224,260,Star Wars: Episode IV - A New Hope,1977
257,296,Pulp Fiction,1994
507,589,Terminator 2: Judgment Day,1991
898,1196,Star Wars: Episode V - The Empire Strikes Back,1980
900,1198,Raiders of the Lost Ark (Indiana Jones and the...,1981
902,1200,Aliens,1986
911,1210,Star Wars: Episode VI - Return of the Jedi,1983
990,1291,Indiana Jones and the Last Crusade,1989
1183,1580,Men in Black (a.k.a. MIB),1997
1298,1732,"Big Lebowski, The",1998
